# 모델 불러오기

In [2]:
!pip install transformers==4.8.2
!pip install tensorflow_addons

import pandas as pd
import numpy as np

import re
import os
import pickle 
import dill # for saving a function as a file
import logging # for changing the tf's logging level
from IPython.display import clear_output 

import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras import layers, initializers, losses, optimizers, metrics 

import transformers
from transformers import TFBertModel 

# 이번 실습에서 추가되었습니다
!pip install sentencepiece==0.1.96
import sentencepiece as spm 
!git clone https://github.com/monologg/KoBERT-Transformers.git 
!mv KoBERT-Transformers/kobert_transformers/tokenization_kobert.py /content 
from tokenization_kobert import KoBertTokenizer 

# Tensorflow logging level 변경 
tf.get_logger().setLevel(logging.ERROR)

# clear the output after the installation
clear_output() 

In [3]:
import datetime
from numba import jit

In [4]:
from google.colab import drive
drive.mount('/gdrive')

data_path = '/gdrive/MyDrive/colab_data/temp_data/' 


# 1) Load the Model-builder function
with open(data_path + 'model_koBERTfunction_v1.pkl', 'rb') as f: # 이번 실습에서 변경되었습니다
    create_model = dill.loads(pickle.load(f)) # use dill to pickle a python function

# 2) Load the Bert-tokenizer 
tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')

# 3) Create the model & load the Model-weights (from checkpoint file)
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.TPUStrategy(resolver) # Obsolete : tf.distribute.experimental.TPUStrategy()

with strategy.scope(): 
    model = create_model(max_length=64) 

checkpoint_path = '/gdrive/MyDrive/colab_data/temp_data/saved_models/'
model.load_weights(filepath=checkpoint_path + 'best_kobert_weights.h5') # 이번 실습에서 변경되었습니다

Mounted at /gdrive


Downloading:   0%|          | 0.00/371k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/77.8k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


Downloading:   0%|          | 0.00/426 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/369M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertModel.

All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [5]:
def encoding(row):
    sentence = row['new'] # title encoding -> new
    SEQ_LEN = 64 # 최대 token 개수 이상의 값으로 임의로 설정

    # Tokenizing / Tokens to sequence numbers / Padding
    encoded_dict = tokenizer.encode_plus(text=re.sub("[^\s0-9a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣]", "", sentence),
                                         padding='max_length', 
                                         truncation = True,
                                         max_length=SEQ_LEN) # SEQ_LEN == 128
    
    token_ids = np.array(encoded_dict['input_ids']).reshape(1, -1) # shape == (1, 128) : like appending to a list 
    token_masks = np.array(encoded_dict['attention_mask']).reshape(1, -1)
    token_segments = np.array(encoded_dict['token_type_ids']).reshape(1, -1)
    new_inputs = (token_ids, token_masks, token_segments)
    return new_inputs

In [6]:

def predict_sentiment(row):
    encode_sentence = row['encode']
    # Prediction
    prediction = model.predict(encode_sentence)
    predicted_probability = np.round(np.max(prediction) * 100, 2) # ex) [[0.0125517 0.9874483]] -> round(0.9874483 * 100, 2) -> round(98.74483, 2) -> 98.74
    predicted_class = ['부정', '긍정'][np.argmax(prediction, axis=1)[0]] # ex) ['부정', '긍정'][[1][0]] -> ['부정', '긍정'][1] -> '긍정'
    
    #print("{}% 확률로 {} 리뷰입니다.".format(predicted_probability, predicted_class))
    return predicted_class, predicted_probability

# 구글드라이브에서 뉴스 크롤링 데이터 가져오기

In [7]:
data_path = '/gdrive/MyDrive/colab_data/news_crawling_v3.csv' 

In [8]:
news_df = pd.read_csv(data_path)
news_df.head()

,Unnamed: 0,Title,Date
0,0,[Asia마감] 中 '나홀로 돈풀기'에 환호…상하이 1.6%↑,2022년05월20일(금)
1,1,"[마켓뷰] 코스피, 국내외 기관 매수에 1.8% 상승 마감...개인 1조 ‘팔자’",2022년05월20일(금)
2,2,"[외환마감]강달러 진정+위안화 강세…환율, 10원 가량 내려 1260원대",2022년05월20일(금)
3,3,"[마감시황]코스피, 하루 만에 2600선 회복…'바이든 효과'",2022년05월20일(금)
4,4,"[시황종합] 코스피, 한미정상회담 수혜 기대감에 상승…2639.29",2022년05월20일(금)


In [9]:
news_df.loc[1,'Title']

'[마켓뷰] 코스피, 국내외 기관 매수에 1.8% 상승 마감...개인 1조 ‘팔자’'

# 헤드라인에서 대괄호 안의 문자 제거

In [10]:
import re
def delete_bracket(row):
  x = row['Title']
  pattern = r'\[([^]]+)\]'
  #x = '이건 [괄호 안의 불필요한 정보를] 삭제하는 코드다.' test code

  text = re.sub(pattern=pattern, repl='', string= x)
  return text


In [11]:

news_df['new'] = news_df.apply(delete_bracket, axis='columns')

In [12]:
news_df

,Unnamed: 0,Title,Date,new
0,0,[Asia마감] 中 '나홀로 돈풀기'에 환호…상하이 1.6%↑,2022년05월20일(금),中 '나홀로 돈풀기'에 환호…상하이 1.6%↑
1,1,"[마켓뷰] 코스피, 국내외 기관 매수에 1.8% 상승 마감...개인 1조 ‘팔자’",2022년05월20일(금),"코스피, 국내외 기관 매수에 1.8% 상승 마감...개인 1조 ‘팔자’"
2,2,"[외환마감]강달러 진정+위안화 강세…환율, 10원 가량 내려 1260원대",2022년05월20일(금),"강달러 진정+위안화 강세…환율, 10원 가량 내려 1260원대"
3,3,"[마감시황]코스피, 하루 만에 2600선 회복…'바이든 효과'",2022년05월20일(금),"코스피, 하루 만에 2600선 회복…'바이든 효과'"
4,4,"[시황종합] 코스피, 한미정상회담 수혜 기대감에 상승…2639.29",2022년05월20일(금),"코스피, 한미정상회담 수혜 기대감에 상승…2639.29"
...,...,...,...,...
35080,35080,[원포인트]5년차 상승국면에서 부각될 주식,2007년01월05일(금),5년차 상승국면에서 부각될 주식
35081,35081,코스닥시장 4대 트렌드에 주목하라,2007년01월03일(수),코스닥시장 4대 트렌드에 주목하라
35082,35082,"올해 인터넷株 투자 ""이것이 핵심""",2007년01월03일(수),"올해 인터넷株 투자 ""이것이 핵심"""
35083,35083,외인은 1월에 산다,2007년01월03일(수),외인은 1월에 산다


# predict

In [13]:
news_df['encode'] = news_df.apply(encoding, axis='columns')

In [ ]:
news_df['label'], news_df['percent'] = news_df.apply(predict_sentiment, axis='columns')

INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond/Identity:0' shape=(None, 64) dtype=int64>, <tf.Tensor 'cond/Identity_8:0' shape=(None, 64) dtype=int64>, <tf.Tensor 'cond/Identity_16:0' shape=(None, 64) dtype=int64>]


In [1]:
news_df

NameError: ignored

#test

In [34]:
temp = news_df[:100]
temp

,Unnamed: 0,Title,Date,new,encode
0,0,[Asia마감] 中 '나홀로 돈풀기'에 환호…상하이 1.6%↑,2022년05월20일(금),中 '나홀로 돈풀기'에 환호…상하이 1.6%↑,"([[2, 1370, 7932, 6079, 1730, 7747, 5579, 5136..."
1,1,"[마켓뷰] 코스피, 국내외 기관 매수에 1.8% 상승 마감...개인 1조 ‘팔자’",2022년05월20일(금),"코스피, 국내외 기관 매수에 1.8% 상승 마감...개인 1조 ‘팔자’","([[2, 4672, 1140, 1261, 1994, 6896, 547, 2670,..."
2,2,"[외환마감]강달러 진정+위안화 강세…환율, 10원 가량 내려 1260원대",2022년05월20일(금),"강달러 진정+위안화 강세…환율, 10원 가량 내려 1260원대","([[2, 807, 5795, 4360, 7227, 7050, 7941, 814, ..."
3,3,"[마감시황]코스피, 하루 만에 2600선 회복…'바이든 효과'",2022년05월20일(금),"코스피, 하루 만에 2600선 회복…'바이든 효과'","([[2, 4672, 4937, 1946, 553, 191, 6559, 5155, ..."
4,4,"[시황종합] 코스피, 한미정상회담 수혜 기대감에 상승…2639.29",2022년05월20일(금),"코스피, 한미정상회담 수혜 기대감에 상승…2639.29","([[2, 4672, 4955, 6255, 7227, 6527, 7954, 2910..."
...,...,...,...,...,...
95,95,예상 웃돈 CPI에 나스닥 -3.18% 급락 [데일리 국제금융시장],2022년05월12일(목),예상 웃돈 CPI에 나스닥 -3.18% 급락,"([[2, 3405, 3528, 5866, 638, 337, 6896, 1370, ..."
96,96,[유럽증시] 미 물가 오름폭 둔화에 동반 상승,2022년05월12일(목),미 물가 오름폭 둔화에 동반 상승,"([[2, 2149, 2136, 3417, 6117, 7734, 1778, 6896..."
97,97,"[마켓뷰] 코스피, 美 CPI 발표 앞두고 하락 마감…외인·기관 ‘팔자’",2022년05월11일(수),"코스피, 美 CPI 발표 앞두고 하락 마감…외인·기관 ‘팔자’","([[2, 4672, 638, 337, 2251, 3185, 4933, 1908, ..."
98,98,'매도→관망' 분위기 반전…美관세 완화에 설레는 中[Asia 마감],2022년05월11일(수),'매도→관망' 분위기 반전…美관세 완화에 설레는 中,"([[2, 1990, 5474, 6165, 2482, 2219, 5474, 6579..."


In [37]:
start=datetime.datetime.now()
temp['label'], temp['percent']  = zip(*temp.apply(predict_sentiment, axis='columns'))
end=datetime.datetime.now()
print(end-start)

0:01:13.552580


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [36]:
start=datetime.datetime.now()
for i in range(100):
  sentence = news_df.loc[i,'Title']
  predicted_class, predicted_probability = predict_sentiment(sentence)
  news_df.loc[i,'label'] = predicted_class
  news_df.loc[i,'percent'] = predicted_probability
end=datetime.datetime.now()
print(end-start)

TypeError: ignored

In [38]:
temp[:20]

,Unnamed: 0,Title,Date,new,encode,label,percent
0,0,[Asia마감] 中 '나홀로 돈풀기'에 환호…상하이 1.6%↑,2022년05월20일(금),中 '나홀로 돈풀기'에 환호…상하이 1.6%↑,"([[2, 1370, 7932, 6079, 1730, 7747, 5579, 5136...",부정,71.32
1,1,"[마켓뷰] 코스피, 국내외 기관 매수에 1.8% 상승 마감...개인 1조 ‘팔자’",2022년05월20일(금),"코스피, 국내외 기관 매수에 1.8% 상승 마감...개인 1조 ‘팔자’","([[2, 4672, 1140, 1261, 1994, 6896, 547, 2670,...",부정,82.65
2,2,"[외환마감]강달러 진정+위안화 강세…환율, 10원 가량 내려 1260원대",2022년05월20일(금),"강달러 진정+위안화 강세…환율, 10원 가량 내려 1260원대","([[2, 807, 5795, 4360, 7227, 7050, 7941, 814, ...",부정,60.08
3,3,"[마감시황]코스피, 하루 만에 2600선 회복…'바이든 효과'",2022년05월20일(금),"코스피, 하루 만에 2600선 회복…'바이든 효과'","([[2, 4672, 4937, 1946, 553, 191, 6559, 5155, ...",긍정,70.98
4,4,"[시황종합] 코스피, 한미정상회담 수혜 기대감에 상승…2639.29",2022년05월20일(금),"코스피, 한미정상회담 수혜 기대감에 상승…2639.29","([[2, 4672, 4955, 6255, 7227, 6527, 7954, 2910...",긍정,75.46
5,5,[코스닥 마감]바이든 방한 훈풍에 1.8%대 상승…880선 눈앞,2022년05월20일(금),바이든 방한 훈풍에 1.8%대 상승…880선 눈앞,"([[2, 2186, 7096, 5928, 2267, 7828, 517, 7970,...",긍정,79.92
6,6,[오전시황] 기관 매수세에 코스피 2600선 회복…개인은 ‘팔자’,2022년05월20일(금),기관 매수세에 코스피 2600선 회복…개인은 ‘팔자’,"([[2, 1261, 1994, 6579, 6896, 4672, 553, 191, ...",부정,52.53
7,7,[굿모닝 증시]美 증시 하락 마감…코스피 기술·리오프닝株 중심 반등할까,2022년05월20일(금),美 증시 하락 마감…코스피 기술·리오프닝株 중심 반등할까,"([[2, 4294, 4933, 1908, 7537, 1289, 6122, 6964...",부정,95.60
8,8,"[외환브리핑]102선으로 떨어진 달러인덱스…환율, 1260원대 하락 시도",2022년05월20일(금),"102선으로 떨어진 달러인덱스…환율, 1260원대 하락 시도","([[2, 533, 119, 6559, 7078, 1860, 1599, 7119, ...",부정,96.06
9,9,"뉴욕증시, 물가·경기 우려에 하락…바이든 오늘 방한 [모닝브리핑]",2022년05월20일(금),"뉴욕증시, 물가·경기 우려에 하락…바이든 오늘 방한","([[2, 1543, 7316, 6705, 2136, 5425, 3499, 6896...",부정,94.81


# 날짜별 평균 감정 지수 구하기



In [ ]:
temp.loc[temp['label'] == '긍정', 'label_index'] = 1
temp.loc[temp['label'] == '부정', 'label_index'] = 0

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [ ]:
groups = temp.groupby('Date')

In [ ]:
groups.mean()

,Unnamed: 0,percent,label_index
Date,,,
2022년05월09일(월),3.0,86.655714,0.000000
2022년05월11일(수),97.5,87.075000,0.000000
2022년05월12일(목),88.5,84.989286,0.142857
2022년05월13일(금),75.0,73.754615,0.615385
2022년05월16일(월),52.0,84.503636,0.272727
2022년05월17일(화),28.0,77.969333,0.266667
2022년05월18일(수),13.5,76.408571,0.500000


In [ ]:
date_list = temp['Date'].unique()

In [ ]:
for i in date_list:
  print(i)

2022년05월09일(월)
2022년05월18일(수)
2022년05월17일(화)
2022년05월16일(월)
2022년05월13일(금)
2022년05월12일(목)
2022년05월11일(수)
